In [1]:
# autoreload libaries
%load_ext autoreload
%autoreload 2

In [ ]:
!python run_update.py

Total Articles to Tag: 2786
Processing Batch 1..
 31%|█████████████▍                             | 16/51 [00:27<01:40,  2.87s/it]

In [2]:
import gspread
import numpy as np
import pandas as pd
import utils
import tqdm

gc = gspread.service_account(filename='../cowid-319502-34c80e63dbfa.json')

RESET_ALL=False
SHEET_KEY = "1FYR8EzZ6xCHIUfYZuw72fRRs5heX2a9gozYkTHR1akA"

UPDATE_BATCH_SIZE = 50

In [3]:
sh = gc.open_by_key(SHEET_KEY)
worksheet = sh.sheet1
data =  worksheet.get_all_values()
data = np.array(data)
articles = pd.DataFrame(data=data[1:,:],columns=data[0,:])
tech_tag = worksheet.findall("Technical (Y/N)", in_row=1)[0]
mappings = utils.load_mappings("mapping.yaml")

TECHNICAL_TAG_COL = tech_tag.col
TECHNICAL_TAG_ADDR = tech_tag.address[0]

In [4]:
if RESET_ALL is False:
    articles = articles[articles["Technical (Y/N)"]==""]

In [5]:
num_batches = int(len(articles)/UPDATE_BATCH_SIZE)

In [9]:
cnt = 0
for article_batch in np.array_split(articles, num_batches):
    cnt+=1
    print(f'Processing Batch {cnt}..')
    categories = utils.get_category_parallel(article_batch["Resource URL"].to_numpy())
    print(categories)
    update_range = []
    for i, idx in enumerate(article_batch.index):
        article = article_batch.loc[idx]
        tag = utils.find_tag(mappings, utils.get_host(article["Resource URL"]), categories[i])
        if tag =="technical":
            tag = "Y"
        elif tag == "non-technical":
            tag = "N"
        else:
            tag = "M"
        article_dict = {
            "values": [[tag]] , 
            "range": TECHNICAL_TAG_ADDR+str(idx+2)
        }
        update_range.append(article_dict)
    if (len(update_range)>0):
        print(f'Updating Batch {cnt} to Google Sheet..')
        worksheet.batch_update(update_range)



  0%|          | 0/51 [00:00<?, ?it/s]

Processing Batch 1..


 37%|███▋      | 19/51 [01:10<01:59,  3.73s/it]


  8%|▊         | 4/51 [00:01<00:22,  2.13it/s]

 12%|█▏        | 6/51 [00:02<00:22,  2.02it/s]

 16%|█▌        | 8/51 [00:03<00:19,  2.18it/s]

 20%|█▉        | 10/51 [00:05<00:26,  1.57it/s]

 24%|██▎       | 12/51 [00:09<00:39,  1.01s/it]

 27%|██▋       | 14/51 [00:11<00:37,  1.02s/it]

 31%|███▏      | 16/51 [00:13<00:36,  1.03s/it]

 35%|███▌      | 18/51 [00:16<00:36,  1.09s/it]

 39%|███▉      | 20/51 [00:18<00:35,  1.15s/it]

KeyboardInterrupt: 